# Open AI -mallien hienosäätö

Tämä muistikirja perustuu Open AI:n nykyisiin ohjeisiin, jotka löytyvät [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) -dokumentaatiosta.

Hienosäätö parantaa perustamallien suorituskykyä sovelluksessasi uudelleenkouluttamalla mallia lisätyllä datalla ja kontekstilla, joka liittyy kyseiseen käyttötapaukseen tai tilanteeseen. Huomaa, että kehotteiden suunnittelutekniikat kuten _few shot learning_ ja _retrieval augmented generation_ mahdollistavat oletuskehotteen parantamisen asiaankuuluvalla datalla laadun parantamiseksi. Näillä lähestymistavoilla on kuitenkin rajoitus kohdemallin maksimimerkkimäärän suhteen.

Hienosäädöllä koulutamme mallia käytännössä uudelleen tarvittavalla datalla (jolloin voimme käyttää paljon enemmän esimerkkejä kuin mitä mahtuu maksimimerkkimäärään) – ja otamme käyttöön _räätälöidyn_ version mallista, joka ei enää tarvitse esimerkkejä syötteen yhteydessä. Tämä parantaa paitsi kehotteen suunnittelun tehokkuutta (meillä on enemmän joustavuutta käyttää merkkimäärää muihin asioihin) myös mahdollisesti alentaa kustannuksia (vähentämällä mallille lähetettävien merkkien määrää syötteen yhteydessä).

Hienosäädössä on 4 vaihetta:
1. Valmistele koulutusdata ja lataa se.
1. Suorita koulutusprosessi saadaksesi hienosäädetyn mallin.
1. Arvioi hienosäädetty malli ja toista laadun parantamiseksi.
1. Ota hienosäädetty malli käyttöön ennustettaessa, kun olet tyytyväinen.

Huomaa, että kaikki perustamallit eivät tue hienosäätöä – [tarkista OpenAI:n dokumentaatiosta](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) viimeisimmät tiedot. Voit myös hienosäätää aiemmin hienosäädettyä mallia. Tässä opetusohjelmassa käytämme kohdemallina hienosäätöön `gpt-35-turbo` -mallia.

---


### Vaihe 1.1: Valmistele aineistosi

Rakennetaan chatbot, joka auttaa sinua ymmärtämään alkuaineiden jaksollista järjestelmää vastaamalla kysymyksiin alkuaineesta limerikin muodossa. Tässä yksinkertaisessa opetusohjelmassa luomme vain aineiston, jolla koulutamme mallin muutamilla esimerkkivastauksilla, jotka näyttävät odotetun tietomuodon. Todellisessa käyttötapauksessa sinun täytyy luoda aineisto paljon suuremmalla määrällä esimerkkejä. Saatat myös pystyä käyttämään avointa aineistoa (sovellusalueellesi), jos sellainen on olemassa, ja muotoilemaan sen uudelleen hienosäätöä varten.

Koska keskitymme `gpt-35-turbo` -malliin ja haemme yksivaiheista vastausta (chat completion), voimme luoda esimerkkejä käyttäen [tätä ehdotettua muotoa](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), joka vastaa OpenAI:n chat completion -vaatimuksia. Jos odotat monivaiheista keskustelusisältöä, käyttäisit [monivaiheista esimerkkimuotoa](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), joka sisältää `weight`-parametrin ilmaistakseen, mitkä viestit tulisi ottaa mukaan (tai jättää pois) hienosäätöprosessissa.

Käytämme tässä opetusohjelmassa yksinkertaisempaa yksivaiheista muotoa. Aineisto on [jsonl-muodossa](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst), jossa on yksi tietue per rivi, kukin JSON-muotoisena objektina. Alla oleva katkelma näyttää 2 tietuetta esimerkkinä – katso [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) koko esimerkkijoukkoa (10 esimerkkiä), jota käytämme hienosäätöopetuksessa. **Huom:** Jokainen tietue _täytyy_ määritellä yhdelle riville (ei jaettuna useammalle riville kuten tyypillisessä muotoillussa JSON-tiedostossa).

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Todellisessa käyttötapauksessa tarvitset paljon suuremman esimerkkijoukon hyvien tulosten saavuttamiseksi – kompromissi on vastausten laadun ja hienosäädön ajan/kustannusten välillä. Käytämme pientä joukkoa, jotta voimme suorittaa hienosäädön nopeasti ja havainnollistaa prosessia. Katso [tämä OpenAI Cookbook -esimerkki](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) monimutkaisemmasta hienosäätöoppaasta.


---

### Vaihe 1.2 Lataa tietojoukko

Lataa tiedot käyttämällä Files API:a [kuten tässä on kuvattu](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Huomaa, että tämän koodin suorittamiseksi sinun on ensin tehtävä seuraavat vaiheet:
 - Asenna `openai` Python-paketti (varmista, että käytät versiota >=0.28.0 uusimpien ominaisuuksien vuoksi)
 - Aseta `OPENAI_API_KEY`-ympäristömuuttuja OpenAI API -avaimellasi
Lisätietoja saat [Asennusoppaasta](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst), joka on kurssilla tarjottu.

Suorita nyt koodi luodaksesi tiedoston latausta varten paikallisesta JSONL-tiedostostasi.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Vaihe 2.1: Luo hienosäätötyö SDK:lla


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Vaihe 2.2: Tarkista työn tila

Tässä on muutamia asioita, joita voit tehdä `client.fine_tuning.jobs` -rajapinnalla:
- `client.fine_tuning.jobs.list(limit=<n>)` - Listaa viimeiset n hienosäätötyötä
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Hae tietoja tietystä hienosäätötyöstä
- `client.fine_tuning.jobs.cancel(<job_id>)` - Peruuta hienosäätötyö
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Listaa enintään n tapahtumaa työstä
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Prosessin ensimmäinen vaihe on _koulutustiedoston validointi_, jotta varmistetaan, että data on oikeassa muodossa.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Vaihe 2.3: Seuraa tapahtumia edistymisen seuraamiseksi


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Vaihe 2.4: Tarkastele tilaa OpenAI-hallintapaneelissa


Voit myös tarkastella tilaa vierailemalla OpenAI:n verkkosivustolla ja tutkimalla alustan _Fine-tuning_-osiota. Tämä näyttää sinulle nykyisen työn tilan ja antaa myös seurata aiempien työn suorituskertojen historiaa. Tässä kuvakaappauksessa näet, että aiempi suoritus epäonnistui ja toinen suoritus onnistui. Kontekstin vuoksi tämä tapahtui, kun ensimmäisessä suorituksessa käytettiin JSON-tiedostoa, jossa oli väärin muotoiltuja tietueita – kun ne korjattiin, toinen suoritus valmistui onnistuneesti ja teki mallin käyttövalmiiksi.

![Fine-tuning job status](../../../../../translated_images/fi/fine-tuned-model-status.563271727bf7bfba.png)


Voit myös tarkastella tilaviestejä ja mittareita selaamalla alaspäin visuaalisessa kojelaudassa, kuten alla on esitetty:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/fi/fine-tuned-messages-panel.4ed0c2da5ea1313b.png) |  ![Metrics](../../../../../translated_images/fi/fine-tuned-metrics-panel.700d7e4995a65229.png)|


---

### Vaihe 3.1: Hae tunnus ja testaa hienosäädetty malli koodissa


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Vaihe 3.2: Lataa ja testaa hienosäädetty malli Playgroundissa

Voit nyt testata hienosäädettyä mallia kahdella tavalla. Ensinnäkin voit vierailla Playgroundissa ja käyttää Models-pudotusvalikkoa valitaksesi juuri hienosäädetyn mallisi listatuista vaihtoehdoista. Toinen vaihtoehto on käyttää Fine-tuning-paneelissa näkyvää "Playground"-vaihtoehtoa (katso yllä oleva kuvakaappaus), joka avaa seuraavan _vertailunäkymän_, jossa perustason ja hienosäädetyn mallin versiot näkyvät rinnakkain nopeaa arviointia varten.

![Fine-tuning job status](../../../../../translated_images/fi/fine-tuned-playground-compare.56e06f0ad8922016.png)

Täytä yksinkertaisesti järjestelmäkonteksti, jota käytit koulutusdatassasi, ja anna testikysymyksesi. Huomaat, että molemmat puolet päivittyvät identtisellä kontekstilla ja kysymyksellä. Suorita vertailu, niin näet eron niiden vastausten välillä. _Huomaa, miten hienosäädetty malli tuottaa vastauksen siinä muodossa, jonka annoit esimerkeissäsi, kun taas perustason malli noudattaa yksinkertaisesti järjestelmän kehotetta_.

![Fine-tuning job status](../../../../../translated_images/fi/fine-tuned-playground-launch.5a26495c983c6350.png)

Huomaat myös, että vertailu näyttää tokenien määrän kullekin mallille sekä päättelyn suorittamiseen kuluneen ajan. **Tämä esimerkki on yksinkertainen ja tarkoitettu prosessin näyttämiseen, eikä se heijasta todellista datasarjaa tai tilannetta**. Saatat huomata, että molemmissa näytteissä on sama määrä tokeneita (järjestelmäkonteksti ja käyttäjän kehotus ovat identtiset), mutta hienosäädetty malli vie enemmän aikaa päättelyyn (mukautettu malli).

Todellisissa tilanteissa et käytä tällaista leikkiesimerkkiä, vaan hienosäätö tehdään todelliseen dataan (esim. tuoteluettelo asiakaspalvelua varten), jolloin vastausten laatu on paljon ilmeisempi. _Tässä_ kontekstissa vastaavanlaisen vastauslaadun saaminen perustason mallilla vaatii enemmän mukautettua kehotteiden suunnittelua, mikä lisää tokenien käyttöä ja mahdollisesti myös päättelyyn kuluvaa aikaa. _Kokeillaksesi tätä, tutustu OpenAI Cookbookin hienosäätöesimerkkeihin._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Vastuuvapauslauseke**:
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, otathan huomioon, että automaattikäännöksissä saattaa esiintyä virheitä tai epätarkkuuksia. Alkuperäinen asiakirja sen alkuperäiskielellä on virallinen lähde. Tärkeissä asioissa suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa tämän käännöksen käytöstä aiheutuvista väärinymmärryksistä tai tulkinnoista.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
